# Fama-French Risk Factors SML & HML
by Dr Liang Jin

Part of AcF701 Python Sessions: [github.com/drliangjin/mini-python-book](https://github.com/drliangjin/mini-python-book)

Based on the Python example on WRDS by Qingyi Song Drechsler: [Fama-French Factors (Python)](https://wrds-www.wharton.upenn.edu/pages/support/applications/risk-factors-and-industry-benchmarks/fama-french-factors-python/)

### Import external packages

In [ ]:
# import pacakges
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

### Connect to WRDS

In [ ]:
# WRDS Connection
conn = wrds.Connection()

### Retrieving Compustat Data

In [ ]:
# Postgres Query
stmt = """
          SELECT gvkey, datadate, at, pstkl, txditc, pstkrv, seq, pstk
          FROM comp.funda
          WHERE indfmt='INDL' 
          AND datafmt='STD'
          AND popsrc='D'
          AND consol='C'
          AND datadate >= '01/01/1959'
       """
comp = conn.raw_sql(stmt)

In [ ]:
# basic info on the data
comp.info()

# a closer look at the data
comp.describe()

# head and tail
comp.head()

### Work on Compustat Data

In [ ]:
# set date and time to the standard format recognised by Pandas and other packages
comp['datadate']=pd.to_datetime(comp['datadate'])

# create a new variable for year
comp['year']=comp['datadate'].dt.year

In [ ]:
# check data
comp.head()

### Deal with prefered stock

In [ ]:
# if pstkrv is missing, then use pstkl
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])

# if created ps is missing, then use pstk
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])

# if ps is still missing, then assign 0
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])

In [ ]:
# again check prefered stock we just created
comp['ps'].describe()

### Book Value of Equity

In [ ]:
# assign 0 to txditc
comp['txditc']=comp['txditc'].fillna(0)

# create a variable, be, for book value of equity
comp['be']=comp['seq']+comp['txditc']-comp['ps']

# if be is missing, replaced by NaN 
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

In [ ]:
# check book value of equity
comp['be'].describe()

### House cleanning

In [ ]:
# sort values so that the dataframe is constructed by id and time
comp=comp.sort_values(by=['gvkey','datadate'])

# count obs? starting from 0 to length of the group -1
comp['count']=comp.groupby(['gvkey']).cumcount()

# house cleanning
comp=comp[['gvkey','datadate','year','be','count']]

In [ ]:
comp.head()

### Retrieving CRSP stock data

In [ ]:
# sql for returning a merged crsp price dataset
stmt = """
          SELECT a.permno, a.permco, a.date, a.ret, a.retx, a.shrout, a.prc,
                 b.shrcd, b.exchcd
          FROM crsp.msf AS a
          LEFT JOIN crsp.msenames AS b
          ON a.permno=b.permno
          AND b.namedt<=a.date
          AND a.date<=b.nameendt
          WHERE a.date BETWEEN '01/01/1959' AND '12/31/2017'
          AND b.exchcd BETWEEN 1 AND 3
       """
crsp_m = conn.raw_sql(stmt)

In [ ]:
# check data
crsp_m.head()

### Data cleanning

In [ ]:
# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# personally, I like to set all the identifications to be strings...because of the annoying trailing zeros

In [ ]:
# format datatime
crsp_m['date']=pd.to_datetime(crsp_m['date'])

# MonthEnd is a function from pandas.tseries.offsets
# convert timestamp to current month end <= for easier merging purpose
# MonthEnd(-1) move backwards by 1 month (last month end)
# MonthEnd(1) next month end
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

### Add delisting return

In [ ]:
# again, sql query
dlret = conn.raw_sql("SELECT permno, dlret, dlstdt FROM crsp.msedelist")

# work on datetime
dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

In [ ]:
# merge two datasets
crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])

# house cleaning
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1
crsp['me']=crsp['prc'].abs()*crsp['shrout'] # calculate market equity
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

### Aggregate market-cap to company level

In [ ]:
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()
# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()

# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])
# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)
# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])
# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

### Work on FF datetime

In [ ]:
# create year and month
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
# keep December market cap
decme=crsp2[crsp2['month']==12]
decme=decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})


In [ ]:
### July to June dates
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

### Stock level characteristics

In [ ]:
# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()
# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)

In [ ]:
# lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

In [ ]:
# baseline me
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

### House cleanning

In [ ]:
# merge result back together
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

decme['year']=decme['year']+1
decme=decme[['permno','year','dec_me']]

# Info as of June
crsp3_jun = crsp3[crsp3['month']==6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])
crsp_jun=crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','retadj','me','wt','cumretx','mebase','lme','dec_me']]
crsp_jun=crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()

### Retrieving CCM data

In [ ]:
ccm=conn.raw_sql("""
                    SELECT gvkey, lpermno AS permno, linktype, linkprim, 
                    linkdt, linkenddt
                    FROM crsp.ccmxpf_linktable
                    WHERE SUBSTR(linktype,1,1)='L'
                    AND (linkprim ='C' OR linkprim='P')
                 """)

In [ ]:
# convert datetime
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

### Merge with Compustat and CRSP

In [ ]:
# left merge on gvkey
ccm1=pd.merge(comp[['gvkey', 'datadate', 'be', 'count']],ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
# create 'jdate' for further merge with crsp dataset
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

In [ ]:
# set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt']) & (ccm1['jdate']<=ccm1['linkenddt'])]
ccm2=ccm2[['gvkey', 'permno', 'datadate', 'yearend', 'jdate', 'be', 'count']]

In [ ]:
# link comp and crsp
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])

# calculate book to market ratio
ccm_jun['beme']=ccm_jun['be']*1000/ccm_jun['dec_me']

### NYSE stock bucket breakdown

In [ ]:
# select NYSE stocks for bucket breakdown
# exchcd = 1 and positive beme and positive me and shrcd in (10,11) and at least 2 years in comp
nyse=ccm_jun[(ccm_jun['exchcd']==1) & 
             (ccm_jun['beme']>0) & 
             (ccm_jun['me']>0) & 
             (ccm_jun['count']>=1) & 
             ((ccm_jun['shrcd']==10) | (ccm_jun['shrcd']==11))]

# size breakdown
nyse_sz=nyse.groupby(['jdate'])['me'].median().to_frame().reset_index().rename(columns={'me':'sizemedn'})

# beme breakdown
nyse_bm=nyse.groupby(['jdate'])['beme'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_bm=nyse_bm[['jdate','30%','70%']].rename(columns={'30%':'bm30', '70%':'bm70'})

# merge two bucket breakdown datasets
nyse_breaks = pd.merge(nyse_sz, nyse_bm, how='inner', on=['jdate'])

# merge back to our main dataset
ccm1_jun = pd.merge(ccm_jun, nyse_breaks, how='left', on=['jdate'])

### Functions for assigning characteristics bucket

In [ ]:
# functions
def sz_bucket(row):
    if row['me']==np.nan:
        value=''
    elif row['me']<=row['sizemedn']:
        value='S'
    else:
        value='B'
    return value

def bm_bucket(row):
    if 0<=row['beme']<=row['bm30']:
        value = 'L'
    elif row['beme']<=row['bm70']:
        value='M'
    elif row['beme']>row['bm70']:
        value='H'
    else:
        value=''
    return value

### Assign size and btm portfolios

In [ ]:
# assign size portfolio
ccm1_jun['szport']=np.where((ccm1_jun['beme']>0) & 
                            (ccm1_jun['me']>0) & 
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(sz_bucket, axis=1), '')

# assign book-to-market portfolio
ccm1_jun['bmport']=np.where((ccm1_jun['beme']>0) & 
                            (ccm1_jun['me']>0) & 
                            (ccm1_jun['count']>=1), 
                            ccm1_jun.apply(bm_bucket, axis=1), '')

# create positivebmeme and nonmissport variable
ccm1_jun['posbm']=np.where((ccm1_jun['beme']>0) & 
                           (ccm1_jun['me']>0) & 
                           (ccm1_jun['count']>=1), 
                           1, 0)
ccm1_jun['nonmissport']=np.where((ccm1_jun['bmport']!=''), 1, 0)

# store portfolio assignment as of June
june=ccm1_jun[['permno','date', 'jdate', 'bmport','szport','posbm','nonmissport']]
june['ffyear']=june['jdate'].dt.year

### Merge with monthly returns

In [ ]:
# merge back with monthly records
crsp3 = crsp3[['date','permno','shrcd','exchcd','retadj',
               'me','wt','cumretx','ffyear','jdate']]
ccm3=pd.merge(crsp3, 
              june[['permno','ffyear','szport','bmport','posbm','nonmissport']], 
              how='left', 
              on=['permno','ffyear'])

In [ ]:
# keeping only records that meet the criteria
ccm4=ccm3[(ccm3['wt']>0) & 
          (ccm3['posbm']==1) &
          (ccm3['nonmissport']==1) & 
          ((ccm3['shrcd']==10) | (ccm3['shrcd']==11))]

### Value-weighted returns

In [ ]:
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

In [ ]:
# value-weigthed return
vwret=ccm4.groupby(['jdate','szport','bmport']).apply(wavg, 'retadj', 'wt').to_frame().reset_index().rename(columns={0: 'vwret'})
vwret['sbport']=vwret['szport']+vwret['bmport'] # <= concat string

# firm count
vwret_n=ccm4.groupby(['jdate','szport','bmport'])['retadj'].count().reset_index().rename(columns={'retadj':'n_firms'})
vwret_n['sbport']=vwret_n['szport']+vwret_n['bmport']

ff_factors=vwret.pivot(index='jdate', columns='sbport', values='vwret').reset_index()
ff_nfirms=vwret_n.pivot(index='jdate', columns='sbport', values='n_firms').reset_index()

### Create SMB and HML factors

In [ ]:
# create SMB and HML factors
ff_factors['WH']=(ff_factors['BH']+ff_factors['SH'])/2
ff_factors['WL']=(ff_factors['BL']+ff_factors['SL'])/2
ff_factors['WHML'] = ff_factors['WH']-ff_factors['WL']

ff_factors['WB']=(ff_factors['BL']+ff_factors['BM']+ff_factors['BH'])/3
ff_factors['WS']=(ff_factors['SL']+ff_factors['SM']+ff_factors['SH'])/3
ff_factors['WSMB'] = ff_factors['WS']-ff_factors['WB']
ff_factors=ff_factors.rename(columns={'jdate':'date'})

# n firm count
ff_nfirms['H']=ff_nfirms['SH']+ff_nfirms['BH']
ff_nfirms['L']=ff_nfirms['SL']+ff_nfirms['BL']
ff_nfirms['HML']=ff_nfirms['H']+ff_nfirms['L']

ff_nfirms['B']=ff_nfirms['BL']+ff_nfirms['BM']+ff_nfirms['BH']
ff_nfirms['S']=ff_nfirms['SL']+ff_nfirms['SM']+ff_nfirms['SH']
ff_nfirms['SMB']=ff_nfirms['B']+ff_nfirms['S']
ff_nfirms['TOTAL']=ff_nfirms['SMB']
ff_nfirms=ff_nfirms.rename(columns={'jdate':'date'})

### Compare with FF

In [ ]:
# download data from wrds
_ff = conn.get_table(library='ff', table='factors_monthly')
_ff=_ff[['date','smb','hml']]
_ff['date']=_ff['date']+MonthEnd(0)

# correlation between our created FF factors and published FF
_ffcomp = pd.merge(_ff, ff_factors[['date','WSMB','WHML']], how='inner', on=['date'])
_ffcomp70=_ffcomp[_ffcomp['date']>='01/01/1970']
print(stats.pearsonr(_ffcomp70['smb'], _ffcomp70['WSMB']))
print(stats.pearsonr(_ffcomp70['hml'], _ffcomp70['WHML']))